In [1]:
from ultralytics import YOLO
import json
import os

# Load a COCO-pretrained YOLO11n model
model = YOLO("yolo11n.pt")

In [2]:
print(model)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_

In [3]:
# Load JSON annotation file
# Define paths
json_path = "annotations.json"  # JSON file path
images_dir = "images"  # Image folder
labels_dir = "labels"  # Output labels directory

os.makedirs(labels_dir, exist_ok=True)

# Load JSON annotation file
with open(json_path, "r") as f:
    data = json.load(f)

# Extract filenames and bounding boxes
file_names = data["train"]["file_names"]
rois_list = data["train"]["rois_list"]

for file_name, rois in zip(file_names, rois_list):
    label_file = os.path.join(labels_dir, file_name.replace(".JPG", ".txt"))

    with open(label_file, "w") as lf:
        for obj in rois:  # Each object has 4 bounding box points
            x_values = [p[0] for p in obj]
            y_values = [p[1] for p in obj]

            x_center = sum(x_values) / len(x_values)
            y_center = sum(y_values) / len(y_values)
            width = max(x_values) - min(x_values)
            height = max(y_values) - min(y_values)

            class_id = 0  # Modify this if multiple classes exist

            lf.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

print("YOLO annotation conversion complete.")

YOLO annotation conversion complete.


In [4]:
# Train with all images
results = model.train(data="././data.yml", epochs=50, imgsz=640)

# Change path in C:\Users\[user]\AppData\Roaming\Ultralytics\settings.json to where the "ML Model" folder is stored on your computer

New https://pypi.org/project/ultralytics/8.3.81 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.80  Python-3.13.2 torch-2.6.0+cpu CPU (Intel Core(TM) i7-8700K 3.70GHz)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=././data.yml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train16, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, 

100%|██████████| 755k/755k [00:00<00:00, 802kB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     


  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256,

train: Scanning C:\Users\Aman.DESKTOP-VEB4T1B\OneDrive\Documents\Rutgers\Capstone\ece-capstone\ML Model\labels... 231 images, 62 backgrounds, 0 corrupt: 100%|██████████| 293/293 [00:04<00:00, 58.72it/s] 

...

      50/50         0G       1.09     0.8252      1.048        135        640: 100%|██████████| 19/19 [02:13<00:00,  7.02s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:44<00:00,  4.48s/it]

                   all        293       7842      0.818      0.711      0.825      0.586



50 epochs completed in 2.931 hours.
Optimizer stripped from runs\detect\train16\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train16\weights\best.pt, 5.5MB

Validating runs\detect\train16\weights\best.pt...
Ultralytics 8.3.80  Python-3.13.2 torch-2.6.0+cpu CPU (Intel Core(TM) i7-8700K 3.70GHz)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:36<00:00,  3.66s/it]


                   all        293       7842      0.818      0.717      0.826      0.586
Speed: 1.6ms preprocess, 55.3ms inference, 0.0ms loss, 11.6ms postprocess per image
Results saved to runs\detect\train16
